In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
#for databricks
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine
import os
from os import listdir
from os.path import isfile, join
import re
import zipfile
import pyspark
from pyspark.sql import SparkSession

In [3]:
#Create SparkSession
spark = SparkSession.builder.master("local[1]").appName("data-gurus").getOrCreate()
print(spark)

In [4]:
#file 0
def AnlagenEegSolar_columns():
    my_dict = {'EegMaStRNummer': [],
               'Registrierungsdatum': [],
               'Zuschlagsnummer': [],
               'MieterstromErsteZuordnungZuschlag': [],
               'InstallierteLeistung': [],
               'AusschreibungZuschlag': [],
               'AnlagenkennzifferAnlagenregister_nv': [],
               'ZugeordneteGebotsmenge': [],
               'AnlageBetriebsstatus': [],
               'DatumLetzteAktualisierung': [],
               'RegistrierungsnummerPvMeldeportal_nv': [],
               'VerknuepfteEinheitenMaStRNummern': [],
               'MieterstromMeldedatum': [],
               'EegInbetriebnahmedatum': [],
               'AnlagenschluesselEeg': [],
               'InanspruchnahmeZahlungNachEeg': [],
               'AnlagenkennzifferAnlagenregister': [],
               'MieterstromZugeordnet': [],
               'RegistrierungsnummerPvMeldeportal': []}
    return my_dict

def AnlagenEegSolar_datatypes_change(df):

    # df['EegMaStRNummer'] = pd.to_str(df['EegMaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    # df['Zuschlagsnummer'] = pd.to_datetime(df['Zuschlagsnummer'])
    df['MieterstromErsteZuordnungZuschlag'] = pd.to_datetime(df['MieterstromErsteZuordnungZuschlag'])
    df['InstallierteLeistung'] = pd.to_numeric(df['InstallierteLeistung'], errors='coerce')
    df['AusschreibungZuschlag'] = pd.to_numeric(df['AusschreibungZuschlag'], errors='coerce')
    df['AnlagenkennzifferAnlagenregister_nv'] = pd.to_numeric(df['AnlagenkennzifferAnlagenregister_nv'])
    df['ZugeordneteGebotsmenge'] = pd.to_numeric(df['ZugeordneteGebotsmenge'], errors='coerce')
    df['AnlageBetriebsstatus'] = pd.to_numeric(df['AnlageBetriebsstatus'], errors='coerce')
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    df['RegistrierungsnummerPvMeldeportal_nv'] = pd.to_numeric(df['RegistrierungsnummerPvMeldeportal_nv'],
                                                               errors='coerce')
    # df['VerknuepfteEinheitenMaStRNummern'] = pd.to_str(df['VerknuepfteEinheitenMaStRNummern'])
    df['MieterstromMeldedatum'] = pd.to_datetime(df['MieterstromMeldedatum'])
    df['EegInbetriebnahmedatum'] = pd.to_datetime(df['EegInbetriebnahmedatum'])
    # df['AnlagenschluesselEeg'] = pd.to_str(df['AnlagenschluesselEeg'])
    df['InanspruchnahmeZahlungNachEeg'] = pd.to_numeric(df['InanspruchnahmeZahlungNachEeg'], errors='coerce')
    # df['AnlagenkennzifferAnlagenregister'] = pd.to_str(df['AnlagenkennzifferAnlagenregister'])
    df['MieterstromZugeordnet'] = pd.to_numeric(df['MieterstromZugeordnet'], errors='coerce')
    # df['RegistrierungsnummerPvMeldeportal'] = pd.to_str(df['RegistrierungsnummerPvMeldeportal'])

In [5]:

#file 1
def AnlagenEegSpeicher_columns():

    my_dict={'EegMaStRNummer':[],
             'Registrierungsdatum':[],
             'EegInbetriebnahmedatum':[],
             'VerknuepfteEinheitenMaStRNummern':[],
             'DatumLetzteAktualisierung':[]
             }
    return my_dict

def AnlagenEegSpeicher_datatypes_change(df):

    #df['EegMaStRNummer'] = pd.to_datetime(df['EegMaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['EegInbetriebnahmedatum'] = pd.to_datetime(df['EegInbetriebnahmedatum'])
    #df['VerknuepfteEinheitenMaStRNummern'] = pd.to_datetime(df['VerknuepfteEinheitenMaStRNummern'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])

In [6]:
#file 2
def AnlagenStromSpeicher_columns():

    my_dict ={'MaStRNummer':[],
              'DatumLetzteAktualisierung':[],
              'NutzbareSpeicherkapazitaet':[],
              'AnlageBetriebsstatus':[],
              'VerknuepfteEinheitenMaStRNummern':[],
              'Registrierungsdatum':[]}
    return my_dict

def AnlagenStromSpeicher_datatypes_change(df):
    df['AnlageBetriebsstatus'] = pd.to_numeric(df['AnlageBetriebsstatus'])
    df['NutzbareSpeicherkapazitaet'] = pd.to_numeric(df['NutzbareSpeicherkapazitaet'])
    # df['VerknuepfteEinheitenMaStRNummern'] = pd.to_datetime(df['VerknuepfteEinheitenMaStRNummern'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    # df['MaStRNummer'] = pd.to_datetime(df['MaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])

In [7]:
#file 3
def EinheitenStromSpeicher_columns():

    my_dict={'DatumLetzteAktualisierung':[],
             'GeplantesInbetriebnahmedatum':[],
             'Energietraeger':[],
             'NameStromerzeugungseinheit':[],
             'Ort':[],
             'Notstromaggregat':[],
             'Breitengrad':[],
             'Kraftwerksnummer_nv':[],
             'Registrierungsdatum':[],
             'Gemeindeschluessel':[],
             'BestandteilGrenzkraftwerk':[],
             'AnlagenbetreiberMastrNummer':[],
             'ZugeordnenteWirkleistungWechselrichter':[],
             'Land':[],
             'WeicDisplayName':[],
             'AcDcKoppelung':[],
             'Laengengrad':[],
             'PumpbetriebKontinuierlichRegelbar':[],
             'GenMastrNummer':[],
             'Gemarkung':[],
             'EinheitSystemstatus':[],
             'EinheitBetriebsstatus':[],
             'Einspeisungsart':[],
             'Hausnummer_nv':[],
             'Postleitzahl':[],
             'NichtVorhandenInMigriertenEinheiten':[],
             'Technologie':[],
             'Kraftwerksnummer':[],
             'Pumpspeichertechnologie':[],
             'Batterietechnologie':[],
             'FernsteuerbarkeitDv':[],
             'NetzbetreiberpruefungStatus':[],
             'DatumEndgueltigeStilllegung':[],
             'DatumDesBetreiberwechsels':[],
             'FernsteuerbarkeitDr':[],
             'DatumBeginnVoruebergehendeStilllegung':[],
             'NetzbetreiberpruefungDatum':[],
             'Weic':[],
             'EegMaStRNummer':[],
             'SpeMastrNummer':[],
             'DatumRegistrierungDesBetreiberwechsels':[],
             'Bundesland':[],
             'AnschlussAnHoechstOderHochSpannung':[],
             'Einsatzort':[],
             'Einsatzverantwortlicher':[],
             'PumpbetriebLeistungsaufnahme':[],
             'EegAnlagentyp':[],
             'LokationMaStRNummer':[],
             'FlurFlurstuecknummern':[],
             'Inbetriebnahmedatum':[],
             'StrasseNichtGefunden':[],
             'Bruttoleistung':[],
             'Adresszusatz':[],
             'Weic_nv':[],
             'Gemeinde':[],
             'Landkreis':[],
             'HausnummerNichtGefunden':[],
             'EinheitMastrNummer':[],
             'Nettonennleistung':[],
             'Hausnummer':[],
             'FernsteuerbarkeitNb':[],
             'DatumWiederaufnahmeBetrieb':[],
             'Strasse':[],
             'NettonennleistungDeutschland':[]}
    return my_dict

def EinheitenStromSpeicher_datatypes_change(df):

    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    df['GeplantesInbetriebnahmedatum'] = pd.to_datetime(df['GeplantesInbetriebnahmedatum'])
    df['Energietraeger'] = pd.to_numeric(df['Energietraeger'])
    #df['NameStromerzeugungseinheit'] = pd.to_numeric(df['NameStromerzeugungseinheit'])
    #df['Ort'] = pd.to_numeric(df['Ort'])
    df['Notstromaggregat'] = pd.to_numeric(df['Notstromaggregat'])
    df['Breitengrad'] = pd.to_numeric(df['Breitengrad'])
    df['Kraftwerksnummer_nv'] = pd.to_numeric(df['Kraftwerksnummer_nv'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['Gemeindeschluessel'] = pd.to_numeric(df['Gemeindeschluessel'])
    df['BestandteilGrenzkraftwerk'] = pd.to_numeric(df['BestandteilGrenzkraftwerk'])
    #df['AnlagenbetreiberMastrNummer'] = pd.to_numeric(df['AnlagenbetreiberMastrNummer'])
    df['ZugeordnenteWirkleistungWechselrichter'] = pd.to_numeric(df['ZugeordnenteWirkleistungWechselrichter'])
    df['Land'] = pd.to_numeric(df['Land'])
    #df['WeicDisplayName'] = pd.to_numeric(df['WeicDisplayName'])
    df['AcDcKoppelung'] = pd.to_numeric(df['AcDcKoppelung'])
    df['Laengengrad'] = pd.to_numeric(df['Laengengrad'])
    df['PumpbetriebKontinuierlichRegelbar'] = pd.to_numeric(df['PumpbetriebKontinuierlichRegelbar'])
    #df['GenMastrNummerv'] = pd.to_numeric(df['GenMastrNummer'])
    #df['Gemarkung'] = pd.to_numeric(df['Gemarkung'])
    df['EinheitSystemstatus'] = pd.to_numeric(df['EinheitSystemstatus'])
    df['EinheitBetriebsstatus'] = pd.to_numeric(df['EinheitBetriebsstatus'])
    df['Einspeisungsart'] = pd.to_numeric(df['Einspeisungsart'])
    df['Hausnummer_nv'] = pd.to_numeric(df['Hausnummer_nv'])
    #df['Postleitzahl'] = pd.to_numeric(df['Postleitzahl'])
    df['NichtVorhandenInMigriertenEinheiten'] = pd.to_numeric(df['NichtVorhandenInMigriertenEinheiten'])
    df['Technologie'] = pd.to_numeric(df['Technologie'])
    #df['Kraftwerksnummer'] = pd.to_numeric(df['Kraftwerksnummer'])
    df['Pumpspeichertechnologie'] = pd.to_numeric(df['Pumpspeichertechnologie'])
    df['Batterietechnologie'] = pd.to_numeric(df['Batterietechnologie'])
    df['FernsteuerbarkeitDv'] = pd.to_numeric(df['FernsteuerbarkeitDv'])
    df['NetzbetreiberpruefungStatus'] = pd.to_numeric(df['NetzbetreiberpruefungStatus'])
    df['DatumEndgueltigeStilllegung'] = pd.to_datetime(df['DatumEndgueltigeStilllegung'], errors='coerce')
    df['DatumDesBetreiberwechsels'] = pd.to_datetime(df['DatumDesBetreiberwechsels'])
    df['FernsteuerbarkeitDr'] = pd.to_numeric(df['FernsteuerbarkeitDr'])
    df['DatumBeginnVoruebergehendeStilllegung'] = pd.to_datetime(df['DatumBeginnVoruebergehendeStilllegung'])
    df['NetzbetreiberpruefungDatum'] = pd.to_datetime(df['NetzbetreiberpruefungDatum'])
    #df['Weic'] = pd.to_numeric(df['Weic'])
    #df['EegMaStRNummer'] = pd.to_numeric(df['EegMaStRNummer'])
    #df['SpeMastrNummer'] = pd.to_numeric(df['SpeMastrNummer'])
    df['DatumRegistrierungDesBetreiberwechsels'] = pd.to_datetime(df['DatumRegistrierungDesBetreiberwechsels'])
    df['Bundesland'] = pd.to_numeric(df['Bundesland'])
    df['AnschlussAnHoechstOderHochSpannung'] = pd.to_numeric(df['AnschlussAnHoechstOderHochSpannung'])
    df['Einsatzort'] = pd.to_numeric(df['Einsatzort'])
    df['Einsatzverantwortlicher'] = pd.to_numeric(df['Einsatzverantwortlicher'])
    df['PumpbetriebLeistungsaufnahme'] = pd.to_numeric(df['PumpbetriebLeistungsaufnahme'])
    df['EegAnlagentyp'] = pd.to_numeric(df['EegAnlagentyp'])
    #df['LokationMaStRNummer'] = pd.to_numeric(df['LokationMaStRNummer'])
    #df['FlurFlurstuecknummern'] = pd.to_numeric(df['FlurFlurstuecknummern'])
    df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'])
    df['StrasseNichtGefunden'] = pd.to_numeric(df['StrasseNichtGefunden'])
    df['Bruttoleistung'] = pd.to_numeric(df['Bruttoleistung'])
    #df['Adresszusatz'] = pd.to_numeric(df['Adresszusatz'])
    df['Weic_nv'] = pd.to_numeric(df['Weic_nv'])
    #df['Gemeinde'] = pd.to_numeric(df['Gemeinde'])
    #df['Landkreis'] = pd.to_numeric(df['Landkreis'])
    #df['HausnummerNichtGefunden'] = pd.to_numeric(df['HausnummerNichtGefunden'])
    #df['EinheitMastrNummer'] = pd.to_numeric(df['EinheitMastrNummer'])
    df['Nettonennleistung'] = pd.to_numeric(df['Nettonennleistung'])
    #df['Hausnummer'] = pd.to_numeric(df['Hausnummer'])
    df['FernsteuerbarkeitNb'] = pd.to_numeric(df['FernsteuerbarkeitNb'])
    df['DatumWiederaufnahmeBetrieb'] = pd.to_datetime(df['DatumWiederaufnahmeBetrieb'])
    #df['Strasse'] = pd.to_numeric(df['Strasse'])
    df['NettonennleistungDeutschland'] = pd.to_numeric(df['NettonennleistungDeutschland'])

In [8]:
#file 4
def EinheitenSolar_columns():
    my_dict={'EinheitMastrNummer':[],
            'DatumLetzteAktualisierung':[],
            'LokationMaStRNummer':[],
            'NetzbetreiberpruefungStatus':[],
            'NetzbetreiberpruefungDatum':[],
            'AnlagenbetreiberMastrNummer':[],
            'Land':[],
            'Bundesland':[],
            'Landkreis':[],
            'Gemeinde':[],
            'Gemeindeschluessel':[],
            'Postleitzahl':[],
            'Ort':[],
            'Registrierungsdatum':[],
            'Inbetriebnahmedatum':[],
            'EinheitSystemstatus':[],
            'EinheitBetriebsstatus':[],
            'NichtVorhandenInMigriertenEinheiten':[],
            'NameStromerzeugungseinheit':[],
            'Weic_nv':[],
            'Kraftwerksnummer_nv':[],
            'Energietraeger':[],
            'Bruttoleistung':[],
            'Nettonennleistung':[],
            'FernsteuerbarkeitNb':[],
            'Einspeisungsart':[],
            'ZugeordneteWirkleistungWechselrichter':[],
            'GemeinsamerWechselrichterMitSpeicher':[],
            'AnzahlModule':[],
            'Lage':[],
            'Leistungsbegrenzung':[],
            'EinheitlicheAusrichtungUndNeigungswinkel':[],
            'Hauptausrichtung':[],
            'HauptausrichtungNeigungswinkel':[],
            'EegMaStRNummer':[],
            'Nutzungsbereich':[],
            'Nebenausrichtung':[],
            'NebenausrichtungNeigungswinkel':[],
            'Gemarkung':[],
            'FlurFlurstuecknummern':[],
            'GeplantesInbetriebnahmedatum':[],
            'Strasse':[],
            'StrasseNichtGefunden':[],
            'Hausnummer':[],
            'Hausnummer_nv':[],
            'HausnummerNichtGefunden':[],
            'Laengengrad':[],
            'Breitengrad':[],
            'InAnspruchGenommeneFlaeche':[],
            'Adresszusatz':[],
            'FernsteuerbarkeitDv':[],
            'FernsteuerbarkeitDr':[],
            'ArtDerFlaecheIds':[],
            'DatumDesBetreiberwechsels':[],
            'DatumRegistrierungDesBetreiberwechsels':[],
            'DatumBeginnVoruebergehendeStilllegung':[],
            'AnschlussAnHoechstOderHochSpannung':[],
            'DatumEndgueltigeStilllegung':[],
            'InAnspruchGenommeneAckerflaeche':[],
            'GenMastrNummer':[],
            'Einsatzverantwortlicher':[],
            'DatumWiederaufnahmeBetrieb':[]}
    
    return my_dict

def EinheitenSolar_datatypes_change(df):

    df['Nebenausrichtung'] = pd.to_numeric(df['Nebenausrichtung'])
    df['Hauptausrichtung'] = pd.to_numeric(df['Hauptausrichtung'])
    #df['Strasse'] = pd.to_numeric(df['Strasse'])
    #df['Gemarkung'] = pd.to_numeric(df['Gemarkung'])
    df['GeplantesInbetriebnahmedatum'] = pd.to_datetime(df['GeplantesInbetriebnahmedatum'])
    df['Nutzungsbereich'] = pd.to_numeric(df['Nutzungsbereich'])
    df['AnzahlModule'] = pd.to_numeric(df['AnzahlModule'])
    df['EinheitBetriebsstatus'] = pd.to_numeric(df['EinheitBetriebsstatus'])
    #df['Einspeisungsart'] = pd.to_numeric(df['Einspeisungsart'])
    df['ZugeordneteWirkleistungWechselrichter'] = pd.to_numeric(df['ZugeordneteWirkleistungWechselrichter'])
    df['InAnspruchGenommeneFlaeche'] = pd.to_numeric(df['InAnspruchGenommeneFlaeche'])
    #df['Gemeindeschluessel'] = pd.to_numeric(df['Gemeindeschluessel'])
    #df['LokationMaStRNummer'] = pd.to_numeric(df['LokationMaStRNummer'])
    df['AnschlussAnHoechstOderHochSpannung'] = pd.to_numeric(df['AnschlussAnHoechstOderHochSpannung'])
    df['DatumWiederaufnahmeBetrieb'] = pd.to_datetime(df['DatumWiederaufnahmeBetrieb'])
    #df['GenMastrNummer'] = pd.to_numeric(df['GenMastrNummer'])
    #df['Gemeinde'] = pd.to_numeric(df['Gemeinde'])
    df['DatumEndgueltigeStilllegung'] = pd.to_datetime(df['DatumEndgueltigeStilllegung'])
    #df['NameStromerzeugungseinheit'] = pd.to_numeric(df['NameStromerzeugungseinheit'])
    df['FernsteuerbarkeitNb'] = pd.to_numeric(df['FernsteuerbarkeitNb'])
    df['DatumDesBetreiberwechsels'] = pd.to_datetime(df['DatumDesBetreiberwechsels'], errors='coerce')
    df['Laengengrad'] = pd.to_numeric(df['Laengengrad'])
    df['Breitengrad'] = pd.to_numeric(df['Breitengrad'])
    #df['EinheitMastrNummer'] = pd.to_numeric(df['EinheitMastrNummer'])
    df['Energietraeger'] = pd.to_numeric(df['Energietraeger'])
    #df['Lage'] = pd.to_numeric(df['Lage'])
    df['StrasseNichtGefunden'] = pd.to_numeric(df['StrasseNichtGefunden'])
    df['Postleitzahl'] = pd.to_numeric(df['Postleitzahl'])
    #df['Landkreis'] = pd.to_numeric(df['Landkreis'])
    df['Leistungsbegrenzung'] = pd.to_numeric(df['Leistungsbegrenzung'])
    #df['Adresszusatz'] = pd.to_numeric(df['Adresszusatz'])
    #df['ArtDerFlaecheIds'] = pd.to_numeric(df['ArtDerFlaecheIds'])
    #df['Ort'] = pd.to_numeric(df['Ort'])
    df['NetzbetreiberpruefungDatum'] = pd.to_datetime(df['NetzbetreiberpruefungDatum'])
    #df['AnlagenbetreiberMastrNummer'] = pd.to_numeric(df['AnlagenbetreiberMastrNummer'])
    df['Weic_nv'] = pd.to_numeric(df['Weic_nv'])
    df['GemeinsamerWechselrichterMitSpeicher'] = pd.to_numeric(df['GemeinsamerWechselrichterMitSpeicher'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['HauptausrichtungNeigungswinkel'] = pd.to_numeric(df['HauptausrichtungNeigungswinkel'])
    #df['Hausnummer'] = pd.to_numeric(df['Hausnummer'])
    df['FernsteuerbarkeitDv'] = pd.to_numeric(df['FernsteuerbarkeitDv'])
    df['NebenausrichtungNeigungswinkel'] = pd.to_numeric(df['NebenausrichtungNeigungswinkel'])
    df['EinheitlicheAusrichtungUndNeigungswinkel'] = pd.to_numeric(df['EinheitlicheAusrichtungUndNeigungswinkel'])
    df['InAnspruchGenommeneAckerflaeche'] = pd.to_numeric(df['InAnspruchGenommeneAckerflaeche'])
    df['HausnummerNichtGefunden'] = pd.to_numeric(df['HausnummerNichtGefunden'])
    df['DatumBeginnVoruebergehendeStilllegung'] = pd.to_datetime(df['DatumBeginnVoruebergehendeStilllegung'])
    #df['EegMaStRNummer'] = pd.to_numeric(df['EegMaStRNummer'])
    df['NetzbetreiberpruefungStatus'] = pd.to_numeric(df['NetzbetreiberpruefungStatus'])
    #df['Bundesland'] = pd.to_numeric(df['Bundesland'])
    df['EinheitSystemstatus'] = pd.to_numeric(df['EinheitSystemstatus'])
    df['Hausnummer_nv'] = pd.to_numeric(df['Hausnummer_nv'])
    df['DatumRegistrierungDesBetreiberwechsels'] = pd.to_datetime(df['DatumRegistrierungDesBetreiberwechsels'])
    df['FernsteuerbarkeitDr'] = pd.to_numeric(df['FernsteuerbarkeitDr'])
    #df['Einsatzverantwortlicher'] = pd.to_numeric(df['Einsatzverantwortlicher'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    #df['FlurFlurstuecknummern'] = pd.to_numeric(df['FlurFlurstuecknummern'])
    df['NichtVorhandenInMigriertenEinheiten'] = pd.to_numeric(df['NichtVorhandenInMigriertenEinheiten'])
    df['Kraftwerksnummer_nv'] = pd.to_numeric(df['Kraftwerksnummer_nv'])
    df['Land'] = pd.to_numeric(df['Land'])
    df['Nettonennleistung'] = pd.to_numeric(df['Nettonennleistung'])
    df['Bruttoleistung'] = pd.to_numeric(df['Bruttoleistung'])
    df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'])

In [9]:
# give path of dataset and file name without the number
def files_mask(dataset_path, filename):
    '''
    
    :param dataset_path: provide dataset path of zip folder
    :param filename: file name without extension type
    :return: matching list with the same kind of files
    '''
    # get all files names in the datasetfolder

    zf = zipfile.ZipFile(dataset_path, "r")
    all_files = []
    for file in zf.namelist():
        if file.endswith(".xml"):
            #[5:] removes DATA/ string from the beginning
            all_files.append(file)

    #all_files = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

    regex_pattern = ".*" + filename + ".*"

    match_list = []
    for i in range(0, len(all_files)):
        match = re.findall(regex_pattern, all_files[i])
        if not match:
            # if list is empty
            pass
        else:
            match_list.append(match[0])

    # print("Required Files in the Dataset: ", match_list)
    return match_list

In [10]:
def xml_to_dataframe_converter(dataset_path, xml_file):
    '''
    
    :param dataset_path:zip folder path 
    :param xml_file: full file name with the extension
    :return: 
    '''

    zf = zipfile.ZipFile(dataset_path, "r")

    #if xml_file in zf.namelist():
    xml_file_open = zf.open(xml_file)
    tree = ET.parse(xml_file_open)
    root = tree.getroot()

    # defining the dictionary from respective files
    if ("AnlagenEegSolar" in xml_file):
        my_dict = AnlagenEegSolar_columns()
    elif ("AnlagenEegSpeicher" in xml_file):
        my_dict = AnlagenEegSpeicher_columns()
    elif ("AnlagenStromSpeicher" in xml_file):
        my_dict = AnlagenStromSpeicher_columns()
    elif ("EinheitenStromSpeicher" in xml_file):
        my_dict = EinheitenStromSpeicher_columns()
    elif ("EinheitenSolar" in xml_file):
        my_dict = EinheitenSolar_columns()
    else:
        print("Could not find the file specified.")

    for elem in root:
        for sub_elem in elem:
            # check if key already exists in the dictionary
            if my_dict.__contains__(sub_elem.tag):
                my_dict[sub_elem.tag].append(sub_elem.text)
            # else create the new key and append
            #else:
                # my_dict[sub_elem.tag] = []
                # my_dict[sub_elem.tag].append(sub_elem.text)
                #print("Add this column: ",my_dict[sub_elem.tag])
                #print("Add this column: ", sub_elem.tag)

    df = pd.DataFrame.from_dict(my_dict, orient='index').transpose()

    #changing column data types by calling the function
    if ("AnlagenEegSolar" in xml_file):
        AnlagenEegSolar_datatypes_change(df)
    elif ("AnlagenEegSpeicher" in xml_file):
        AnlagenEegSpeicher_datatypes_change(df)
    elif ("AnlagenStromSpeicher" in xml_file):
        AnlagenStromSpeicher_datatypes_change(df)
    elif ("EinheitenStromSpeicher" in xml_file):
        EinheitenStromSpeicher_datatypes_change(df)
    elif ("EinheitenSolar" in xml_file):
        EinheitenSolar_datatypes_change(df)

    #print(df.dtypes)
    return df

In [11]:
# Spark data dictionary for each file type 
# TODO: please fill this schema dictionary for all files types, and check download
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType, IntegerType
schemaDict = {'anlageneegsolar': StructType([StructField('EegMaStRNummer', StringType(), True), StructField('Registrierungsdatum', TimestampType(), True), StructField('Zuschlagsnummer', StringType(), True), StructField('MieterstromErsteZuordnungZuschlag', TimestampType(), True), StructField('InstallierteLeistung', DoubleType(), True), StructField('AusschreibungZuschlag', DoubleType(), True), StructField('AnlagenkennzifferAnlagenregister_nv', LongType(), True), StructField('ZugeordneteGebotsmenge', DoubleType(), True), StructField('AnlageBetriebsstatus', LongType(), True), StructField('DatumLetzteAktualisierung', TimestampType(), True), StructField('RegistrierungsnummerPvMeldeportal_nv', LongType(), True), StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), StructField('MieterstromMeldedatum', TimestampType(), True), StructField('EegInbetriebnahmedatum', TimestampType(), True), StructField('AnlagenschluesselEeg', StringType(), True), StructField('InanspruchnahmeZahlungNachEeg', DoubleType(), True), StructField('AnlagenkennzifferAnlagenregister', StringType(), True), StructField('MieterstromZugeordnet', DoubleType(), True), StructField('RegistrierungsnummerPvMeldeportal', StringType(), True)]),
              'anlageneegspeicher': StructType([StructField('EegMaStRNummer', StringType(), True), StructField('Registrierungsdatum', TimestampType(), True), StructField('EegInbetriebnahmedatum', TimestampType(), True), StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), StructField('DatumLetzteAktualisierung', TimestampType(), True)]), 
              'einheitenstromspeicher': StructType([StructField('DatumLetzteAktualisierung', TimestampType(), True), StructField('GeplantesInbetriebnahmedatum', TimestampType(), True), StructField('Energietraeger', LongType(), True), StructField('NameStromerzeugungseinheit', StringType(), True), StructField('Ort', StringType(), True), StructField('Notstromaggregat', DoubleType(), True), StructField('Breitengrad', DoubleType(), True), StructField('Kraftwerksnummer_nv', LongType(), True), StructField('Registrierungsdatum', TimestampType(), True), StructField('Gemeindeschluessel', DoubleType(), True), StructField('BestandteilGrenzkraftwerk', DoubleType(), True), StructField('AnlagenbetreiberMastrNummer', StringType(), True), StructField('ZugeordnenteWirkleistungWechselrichter', DoubleType(), True), StructField('Land', LongType(), True), StructField('WeicDisplayName', StringType(), True), StructField('AcDcKoppelung', DoubleType(), True), StructField('Laengengrad', DoubleType(), True), StructField('PumpbetriebKontinuierlichRegelbar', DoubleType(), True), StructField('GenMastrNummer', StringType(), True), StructField('Gemarkung', StringType(), True), StructField('EinheitSystemstatus', LongType(), True), StructField('EinheitBetriebsstatus', LongType(), True), StructField('Einspeisungsart', DoubleType(), True), StructField('Hausnummer_nv', DoubleType(), True), StructField('Postleitzahl', StringType(), True), StructField('NichtVorhandenInMigriertenEinheiten', LongType(), True), StructField('Technologie',StringType(), True), StructField('Kraftwerksnummer', StringType(), True), StructField('Pumpspeichertechnologie', DoubleType(), True), StructField('Batterietechnologie', DoubleType(), True), StructField('FernsteuerbarkeitDv', DoubleType(), True), StructField('NetzbetreiberpruefungStatus', LongType(), True), StructField('DatumEndgueltigeStilllegung', TimestampType(), True), StructField('DatumDesBetreiberwechsels', TimestampType(), True), StructField('FernsteuerbarkeitDr', DoubleType(), True), StructField('DatumBeginnVoruebergehendeStilllegung', TimestampType(), True), StructField('NetzbetreiberpruefungDatum', TimestampType(), True), StructField('Weic', StringType(), True), StructField('EegMaStRNummer', StringType(), True), StructField('SpeMastrNummer', StringType(), True), StructField('DatumRegistrierungDesBetreiberwechsels', TimestampType(), True), StructField('Bundesland', DoubleType(), True), StructField('AnschlussAnHoechstOderHochSpannung', DoubleType(), True), StructField('Einsatzort', DoubleType(), True), StructField('Einsatzverantwortlicher', DoubleType(), True), StructField('PumpbetriebLeistungsaufnahme', DoubleType(), True), StructField('EegAnlagentyp', LongType(), True), StructField('LokationMaStRNummer', StringType(), True), StructField('FlurFlurstuecknummern', StringType(), True), StructField('Inbetriebnahmedatum', TimestampType(), True), StructField('StrasseNichtGefunden', DoubleType(), True), StructField('Bruttoleistung', DoubleType(), True), StructField('Adresszusatz', StringType(), True), StructField('Weic_nv', LongType(), True), StructField('Gemeinde', StringType(), True), StructField('Landkreis', StringType(), True), StructField('HausnummerNichtGefunden', StringType(), True), StructField('EinheitMastrNummer', StringType(), True), StructField('Nettonennleistung', DoubleType(), True), StructField('Hausnummer', StringType(), True), StructField('FernsteuerbarkeitNb', DoubleType(), True), StructField('DatumWiederaufnahmeBetrieb', TimestampType(), True), StructField('Strasse', StringType(), True), StructField('NettonennleistungDeutschland', DoubleType(), True)]),
              'anlagenstromspeicher': StructType([StructField('MaStRNummer', StringType(), True), StructField('DatumLetzteAktualisierung', TimestampType(), True), StructField('NutzbareSpeicherkapazitaet', DoubleType(), True), StructField('AnlageBetriebsstatus', LongType(), True), StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), StructField('Registrierungsdatum', TimestampType(), True)]),
              'einheitensolar': StructType([StructField('EinheitMastrNummer', StringType(), True),
                                            StructField('DatumLetzteAktualisierung', StringType(), True),
                                            StructField('LokationMaStRNummer', StringType(), True),
                                            StructField('NetzbetreiberpruefungStatus', LongType(), True), 
                                            StructField('NetzbetreiberpruefungDatum', TimestampType(), True), 
                                            StructField('AnlagenbetreiberMastrNummer', StringType(), True), 
                                            StructField('Land', LongType(), True), 
                                            StructField('Bundesland', StringType(), True), 
                                            StructField('Landkreis', StringType(), True), 
                                            StructField('Gemeinde', StringType(), True), 
                                            StructField('Gemeindeschluessel', StringType(), True), 
                                            StructField('Postleitzahl', StringType(), True), 
                                            StructField('Ort', StringType(), True), 
                                            StructField('Registrierungsdatum', TimestampType(), True), 
                                            StructField('Inbetriebnahmedatum', StringType(), True), 
                                            StructField('EinheitSystemstatus', LongType(), True), 
                                            StructField('EinheitBetriebsstatus', LongType(), True), 
                                            StructField('NichtVorhandenInMigriertenEinheiten', LongType(), True), 
                                            StructField('NameStromerzeugungseinheit', StringType(), True), 
                                            StructField('Weic_nv', LongType(), True), 
                                            StructField('Kraftwerksnummer_nv', LongType(), True), 
                                            StructField('Energietraeger', LongType(), True), 
                                            StructField('Bruttoleistung', DoubleType(), True), 
                                            StructField('Nettonennleistung', DoubleType(), True), 
                                            StructField('FernsteuerbarkeitNb', DoubleType(), True), 
                                            StructField('Einspeisungsart', StringType(), True), 
                                            StructField('ZugeordneteWirkleistungWechselrichter', DoubleType(), True), 
                                            StructField('GemeinsamerWechselrichterMitSpeicher', DoubleType(), True), 
                                            StructField('AnzahlModule', DoubleType(), True), 
                                            StructField('Lage', StringType(), True), 
                                            StructField('Leistungsbegrenzung', DoubleType(), True), 
                                            StructField('EinheitlicheAusrichtungUndNeigungswinkel', DoubleType(), True), 
                                            StructField('Hauptausrichtung', StringType(), True), 
                                            StructField('HauptausrichtungNeigungswinkel', DoubleType(), True), 
                                            StructField('EegMaStRNummer', StringType(), True), 
                                            StructField('Nutzungsbereich', DoubleType(), True), 
                                            StructField('Nebenausrichtung', DoubleType(), True), 
                                            StructField('NebenausrichtungNeigungswinkel', DoubleType(), True), 
                                            StructField('Gemarkung', StringType(), True), 
                                            StructField('FlurFlurstuecknummern', StringType(), True), 
                                            StructField('GeplantesInbetriebnahmedatum', TimestampType(), True), 
                                            StructField('Strasse', StringType(), True), 
                                            StructField('StrasseNichtGefunden', DoubleType(), True), 
                                            StructField('Hausnummer', StringType(), True), 
                                            StructField('Hausnummer_nv', DoubleType(), True), 
                                            StructField('HausnummerNichtGefunden', DoubleType(), True), 
                                            StructField('Laengengrad', DoubleType(), True), 
                                            StructField('Breitengrad', DoubleType(), True), 
                                            StructField('InAnspruchGenommeneFlaeche', DoubleType(), True), 
                                            StructField('Adresszusatz', StringType(), True), 
                                            StructField('FernsteuerbarkeitDv', DoubleType(), True), 
                                            StructField('FernsteuerbarkeitDr', DoubleType(), True), 
                                            StructField('ArtDerFlaecheIds', StringType(), True), 
                                            StructField('DatumDesBetreiberwechsels', TimestampType(), True), 
                                            StructField('DatumRegistrierungDesBetreiberwechsels', TimestampType(), True), 
                                            StructField('DatumBeginnVoruebergehendeStilllegung', TimestampType(), True), 
                                            StructField('AnschlussAnHoechstOderHochSpannung', DoubleType(), True),
                                            StructField('DatumEndgueltigeStilllegung', TimestampType(), True),
                                            StructField('InAnspruchGenommeneAckerflaeche', DoubleType(), True),
                                            StructField('GenMastrNummer', StringType(), True),
                                            StructField('Einsatzverantwortlicher', StringType(), True),
                                            StructField('DatumWiederaufnahmeBetrieb', TimestampType(), True)])
             }

In [12]:
import warnings
warnings.filterwarnings('ignore') 

#this is the main function
def xml_parser():
    # change dataset path
    #dataset_path = "G:\DATA GURUS WORK\AAAA\"
    dataset_path = "/home/jovyan/work/AAAA.zip"

    #file_names = ['AnlagenEegSolar', 'AnlagenEegSpeicher', 'AnlagenStromSpeicher', 'EinheitenStromSpeicher', 'EinheitenSolar']
    file_names = ['EinheitenSolar']

    for i in range(0, len(file_names)):
    
        file_name = file_names[i]
        
        #dbutils.fs.rm("file:/home/jovyan/spark-warehouse/marktstammdaten_{file_name.lower()}", recurse=True)
        spark.sql(f"drop table if exists marktstammdaten_{file_name.lower()}")

        print("Current File Type : ", file_name)

        xml_files_list = files_mask(dataset_path, file_name)
        print("XML FILES LIST: ", xml_files_list)

        #print(schemaDict[file_name.lower()])      #TODO: delete it
        
        for xml_file in xml_files_list:

            print(f"Processing : {xml_file}")
            data = xml_to_dataframe_converter(dataset_path,xml_file)
            # Create DataFrame
            f = pd.DataFrame(data)
            #creating spark data frame            
            sdf = spark.createDataFrame(data, schema=schemaDict[file_name.lower()] if file_name.lower() in schemaDict.keys() else None)
            
            # change database table name
            table_name = file_name.lower()
            table_name = "marktstammdaten_" + table_name

            try:
                sdf.write.option("path", f"file:/home/jovyan/spark-warehouse/marktstammdaten_{file_name.lower()}").mode("append").saveAsTable(table_name)
            except Exception as e:
                print("Exception occurred")
                print(e)
                
    print("Finished processing!!!!")

In [13]:
xml_parser()

Current File Type :  EinheitenSolar
XML FILES LIST:  ['AAAA/EinheitenSolar_1.xml', 'AAAA/EinheitenSolar_2.xml', 'AAAA/EinheitenSolar_3.xml', 'AAAA/EinheitenSolar_4.xml', 'AAAA/EinheitenSolar_5.xml', 'AAAA/EinheitenSolar_6.xml', 'AAAA/EinheitenSolar_7.xml']
Processing : AAAA/EinheitenSolar_1.xml
Processing : AAAA/EinheitenSolar_2.xml
Processing : AAAA/EinheitenSolar_3.xml
Processing : AAAA/EinheitenSolar_4.xml
Processing : AAAA/EinheitenSolar_5.xml
Processing : AAAA/EinheitenSolar_6.xml
Processing : AAAA/EinheitenSolar_7.xml
Finished processing!!!!


In [14]:
aes = spark.sql("select * from marktstammdaten_einheitensolar")
print(aes.count())
print(aes.schema)
aes.show()

700000
StructType([StructField('EinheitMastrNummer', StringType(), True), StructField('DatumLetzteAktualisierung', StringType(), True), StructField('LokationMaStRNummer', StringType(), True), StructField('NetzbetreiberpruefungStatus', LongType(), True), StructField('NetzbetreiberpruefungDatum', TimestampType(), True), StructField('AnlagenbetreiberMastrNummer', StringType(), True), StructField('Land', LongType(), True), StructField('Bundesland', StringType(), True), StructField('Landkreis', StringType(), True), StructField('Gemeinde', StringType(), True), StructField('Gemeindeschluessel', StringType(), True), StructField('Postleitzahl', StringType(), True), StructField('Ort', StringType(), True), StructField('Registrierungsdatum', TimestampType(), True), StructField('Inbetriebnahmedatum', StringType(), True), StructField('EinheitSystemstatus', LongType(), True), StructField('EinheitBetriebsstatus', LongType(), True), StructField('NichtVorhandenInMigriertenEinheiten', LongType(), True), 

In [16]:
from pyspark.sql.types import IntegerType
spark.sql("DROP TABLE IF EXISTS test_table");
aes = spark.sql("select Bundesland, Gemeindeschluessel, GenMastrNummer from marktstammdaten_einheitensolar")
aes.show()
change_field = aes.withColumn("Bundesland", aes["Bundesland"].cast(IntegerType())).withColumn("Gemeindeschluessel", aes["Gemeindeschluessel"].cast(IntegerType()))
change_field.write.option("path", f"file:/home/jovyan/spark-warehouse/test").mode("append").saveAsTable("test_table")
change_field = spark.sql("select Bundesland, Gemeindeschluessel from test_table")
print( "field.schema : ",change_field.schema)
change_field.show()

+----------+------------------+---------------+
|Bundesland|Gemeindeschluessel| GenMastrNummer|
+----------+------------------+---------------+
|      1412|          10045112|SGE930106700255|
|      1405|          06631007|SGE930724907302|
|      1409|          05754032|SGE978827672451|
|      1403|          09187128|SGE962950074503|
|      1409|          05124000|SGE974236257272|
|      1400|          12065256|SGE908622643226|
|      1405|          06634001|SGE909712265519|
|      1402|          08231000|SGE962101483444|
|      1409|          05358024|SGE919114936047|
|      1415|          16072005|SGE965514230032|
|      1403|          09278154|SGE939942791014|
|      1408|          03405000|SGE962950074503|
|      1411|          01051074|SGE989419375865|
|      1402|          08416041|SGE910899315157|
|      1402|          08115041|SGE976622583804|
|      1403|          09575168|SGE959416782577|
|      1402|          08315107|SGE958762232514|
|      1410|          07232315|SGE911021